In [32]:
from datetime import datetime as dt
import time
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import  os as os
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [34]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [35]:
spacename = input('Enter space name')

In [36]:
daysLimit = int(input('How many days in the past do you want to go?'))

In [37]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == spacename, ##'fuse.eth',
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)


In [38]:
proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.created,
    proposals.id,
    proposals.start,
    proposals.end,
    proposals.votes
])

proposals_snapshots['createdDate']=(pd.to_datetime(proposals_snapshots['proposals_created'],unit='s'))
proposals_snapshots['startDate']=(pd.to_datetime(proposals_snapshots['proposals_start'],unit='s'))
proposals_snapshots['endDate']=(pd.to_datetime(proposals_snapshots['proposals_end'],unit='s'))
proposals_snapshots.head(10)

,proposals_title,proposals_created,proposals_id,proposals_start,proposals_end,proposals_votes,createdDate,startDate,endDate
0,OIP-136: Cross-chain native OHM as bonus proje...,1680108558,0xe8117f18e79acf2e15f7254308c8d1c1138840f5c907...,1680108526,1680367726,74,2023-03-29 16:49:18,2023-03-29 16:48:46,2023-04-01 16:48:46
1,TAP-22 - Treasury Segmentation,1680108259,0xebd3cf4c019ddf8eb637c816379c8b464fc20ee78284...,1680108233,1680367433,63,2023-03-29 16:44:19,2023-03-29 16:43:53,2023-04-01 16:43:53
2,OIP-135: Olympus Grants Program 1-Year Review ...,1679402793,0x246ccd5ab718a160f03e3dbbd1280ebb5389d3637946...,1679402748,1679686380,67,2023-03-21 12:46:33,2023-03-21 12:45:48,2023-03-24 19:33:00
3,AP-23 - Liquidity Incentives for Aura / Balanc...,1679325165,0xde251686e7f312a5e524ba45f4115a887e31ab6a3cb5...,1679325106,1679500800,50,2023-03-20 15:12:45,2023-03-20 15:11:46,2023-03-22 16:00:00
4,OIP 134: Hypernative as a security and risk de...,1679244244,0x58a7bc957b83e4a6424cb72d80bf3c3b0bd8025d0da1...,1679244167,1679503367,68,2023-03-19 16:44:04,2023-03-19 16:42:47,2023-03-22 16:42:47
5,OIP-133: Staking Rate Strategy,1678428395,0x0fbdc3e8adbda28970646743492623e92e3a7c2336d4...,1678428364,1678687564,76,2023-03-10 06:06:35,2023-03-10 06:06:04,2023-03-13 06:06:04
6,OIP-132: wstETH/OHM Boosted Liquidity Engine,1678427799,0x991058cca07d1e6f361217927a99951d200826e2ee57...,1678427767,1678686967,65,2023-03-10 05:56:39,2023-03-10 05:56:07,2023-03-13 05:56:07
7,OIP 131: Boosted Liquidity Engine as a Key Pro...,1678126068,0xe02d0e0d6cb2ad8e6a6512ab1d138d9ec6b64919715a...,1678125946,1678467600,80,2023-03-06 18:07:48,2023-03-06 18:05:46,2023-03-10 17:00:00
8,OIP-130: Lending AMO as a key DAO project for ...,1677691772,0x0e1173ac9585c4e46723d69ef1a4ef1995390d9d3f90...,1677691749,1677950949,55,2023-03-01 17:29:32,2023-03-01 17:29:09,2023-03-04 17:29:09
9,OOP-1 Funding request for 2023 Q1 and Q2 budget.,1677453933,0xadd0850bb53fc31df236d61e2b53326748c80ef72f54...,1677453900,1677713100,60,2023-02-26 23:25:33,2023-02-26 23:25:00,2023-03-01 23:25:00


In [39]:
total_snapshots = len(proposals_snapshots)
print(total_snapshots)

250


In [40]:
proposals_choices = sg.query(proposals.choices)
proposals_choices = pd.DataFrame(proposals_choices)
proposals_choices.head(100)

,0,1,2,3,4,5,6
0,APPROVE OIP-136 bonus project,REJECT OIP-136 bonus project,None,None,None,None,None
1,Approve TAP-22,Reject TAP-22,None,None,None,None,None
2,Approve the changes as specified,Do not approve the changes as sp,None,None,None,None,None
3,Approve Incentive Budget,Reject,None,None,None,None,None
4,For,Against,None,None,None,None,None
...,...,...,...,...,...,...,...
95,YES LETS DO IT!,NO,None,None,None,None,None
96,Yes,No,None,None,None,None,None
97,Yes,No,None,None,None,None,None
98,Yes,No,None,None,None,None,None


proposals_choices = sg.query(proposals.choices)
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7'])
proposals_choices.head(100)

In [41]:
olympus_governance_view = pd.DataFrame()
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)
olympus_governance_view.head(5)

,proposals_title,proposals_created,proposals_id,proposals_start,proposals_end,proposals_votes,createdDate,startDate,endDate,0,1,2,3,4,5,6
0,OIP-136: Cross-chain native OHM as bonus proje...,1680108558,0xe8117f18e79acf2e15f7254308c8d1c1138840f5c907...,1680108526,1680367726,74,2023-03-29 16:49:18,2023-03-29 16:48:46,2023-04-01 16:48:46,APPROVE OIP-136 bonus project,REJECT OIP-136 bonus project,None,None,None,None,None
1,TAP-22 - Treasury Segmentation,1680108259,0xebd3cf4c019ddf8eb637c816379c8b464fc20ee78284...,1680108233,1680367433,63,2023-03-29 16:44:19,2023-03-29 16:43:53,2023-04-01 16:43:53,Approve TAP-22,Reject TAP-22,None,None,None,None,None
2,OIP-135: Olympus Grants Program 1-Year Review ...,1679402793,0x246ccd5ab718a160f03e3dbbd1280ebb5389d3637946...,1679402748,1679686380,67,2023-03-21 12:46:33,2023-03-21 12:45:48,2023-03-24 19:33:00,Approve the changes as specified,Do not approve the changes as sp,None,None,None,None,None
3,AP-23 - Liquidity Incentives for Aura / Balanc...,1679325165,0xde251686e7f312a5e524ba45f4115a887e31ab6a3cb5...,1679325106,1679500800,50,2023-03-20 15:12:45,2023-03-20 15:11:46,2023-03-22 16:00:00,Approve Incentive Budget,Reject,None,None,None,None,None
4,OIP 134: Hypernative as a security and risk de...,1679244244,0x58a7bc957b83e4a6424cb72d80bf3c3b0bd8025d0da1...,1679244167,1679503367,68,2023-03-19 16:44:04,2023-03-19 16:42:47,2023-03-22 16:42:47,For,Against,None,None,None,None,None


In [42]:
len(olympus_governance_view)

250

In [43]:
olympus_governance_view.drop_duplicates()

,proposals_title,proposals_created,proposals_id,proposals_start,proposals_end,proposals_votes,createdDate,startDate,endDate,0,1,2,3,4,5,6
0,OIP-136: Cross-chain native OHM as bonus proje...,1680108558,0xe8117f18e79acf2e15f7254308c8d1c1138840f5c907...,1680108526,1680367726,74,2023-03-29 16:49:18,2023-03-29 16:48:46,2023-04-01 16:48:46,APPROVE OIP-136 bonus project,REJECT OIP-136 bonus project,None,None,None,None,None
1,TAP-22 - Treasury Segmentation,1680108259,0xebd3cf4c019ddf8eb637c816379c8b464fc20ee78284...,1680108233,1680367433,63,2023-03-29 16:44:19,2023-03-29 16:43:53,2023-04-01 16:43:53,Approve TAP-22,Reject TAP-22,None,None,None,None,None
2,OIP-135: Olympus Grants Program 1-Year Review ...,1679402793,0x246ccd5ab718a160f03e3dbbd1280ebb5389d3637946...,1679402748,1679686380,67,2023-03-21 12:46:33,2023-03-21 12:45:48,2023-03-24 19:33:00,Approve the changes as specified,Do not approve the changes as sp,None,None,None,None,None
3,AP-23 - Liquidity Incentives for Aura / Balanc...,1679325165,0xde251686e7f312a5e524ba45f4115a887e31ab6a3cb5...,1679325106,1679500800,50,2023-03-20 15:12:45,2023-03-20 15:11:46,2023-03-22 16:00:00,Approve Incentive Budget,Reject,None,None,None,None,None
4,OIP 134: Hypernative as a security and risk de...,1679244244,0x58a7bc957b83e4a6424cb72d80bf3c3b0bd8025d0da1...,1679244167,1679503367,68,2023-03-19 16:44:04,2023-03-19 16:42:47,2023-03-22 16:42:47,For,Against,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,OIP-5: Lower LP Staking Rewards,1619738500,QmTrwGU99RMqTeeCKy5oW9Le9M2mXHHdNeEWzn7T67watz,1619737200,1619985600,276,2021-04-29 23:21:40,2021-04-29 23:00:00,2021-05-02 20:00:00,For: Option 1,For: Option 2,Against,None,None,None,None
246,Deposit SLP,1618764942,QmZhSNWAzrDy5KiGSeBsKBXUHW6D4vJnfCuqmdai8LPETd,1618761600,1618934400,225,2021-04-18 16:55:42,2021-04-18 16:00:00,2021-04-20 16:00:00,Deposit; keep Onsen,Deposit; no Onsen,Don't deposit,None,None,None,None
247,Loosen Policy,1618763632,QmZJb2evEjsxtLz92XBuhApWty69fVqUpFHzuVjMjCd3Se,1618761600,1618934400,229,2021-04-18 16:33:52,2021-04-18 16:00:00,2021-04-20 16:00:00,Do it,Don't do it,None,None,None,None,None
248,Add DAI Bonds,1618763184,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,1618761600,1618934400,235,2021-04-18 16:26:24,2021-04-18 16:00:00,2021-04-20 16:00:00,Add; remove sales,Add; phase out sales,Add; keep sales,Do not add,None,None,None


In [44]:
len(olympus_governance_view)

250

In [46]:
##this captures the ENTIRE list of people who voted
voteTicker = 0
totalProposals = len(olympus_governance_view)
voteslist = pd.DataFrame()
votesDb = pd.DataFrame()
voteListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 90
datediff=0
epochlimit = (90*86400)
ut = time.time()
limitTimestamp = int(ut - epochlimit)
limitDate = datetime.fromtimestamp(limitTimestamp)
limitDate = limitDate.strftime('%m-%d-%Y')

while int(datediff) < int(daysLimit):
    proposal_id = olympus_governance_view.iloc[voteTicker,2]
    skipValue = (voteTicker)*(1000)
    vote_tracker = snapshot.Query.votes(
        #orderBy = 'created',
        #orderDirection='asc',
        first=1000,
        where=[
          snapshot.Vote.proposal == proposal_id
          #snapshot.Vote.created > limitTimestamp
        ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
    ])


    voting_snapshots['proposals_id']= olympus_governance_view.iloc[voteTicker,2]
    #voteDate = votesDb.iat[voteTicker,4]

    votesDb=pd.concat([voting_snapshots, votesDb])
    votesDb['createdDate']=(pd.to_datetime(votesDb['votes_created'],unit='s'))
    then = votesDb.iat[voteTicker,6]
    now = datetime.now()
    delta = now-then
    datediff = delta.days
    votesDbLength = len(votesDb)
    voteListLength = len(voting_snapshots)
    recordTimestamp1 = votesDb.iat[voteTicker,2]
    recordTimestamp = dt.fromtimestamp( recordTimestamp1 )
    now = (int(dt.utcnow().timestamp()))
    #datediff=abs(int(now) - recordTimestamp1)

    print('ticker', voteTicker, 'proposal',proposal_id, 'records:',voteListLength, 'DB size:',votesDbLength, '    -days ago:', datediff, '     -date', then, '    -exit?', exit )
    #print(proposal_id, voteDate, datediff)
    voteTicker = voteTicker+1

votesDb.drop_duplicates
print("Done. Total Records collected: ", votesDbLength)

ticker 0 proposal 0xe8117f18e79acf2e15f7254308c8d1c1138840f5c907b2e08101800196356cb0 records: 74 DB size: 74     -days ago: 4      -date 2023-04-01 16:18:43     -exit? False
ticker 1 proposal 0xebd3cf4c019ddf8eb637c816379c8b464fc20ee78284a129bbeb1456370f8c0d records: 63 DB size: 137     -days ago: 4      -date 2023-04-01 13:15:20     -exit? False
ticker 2 proposal 0x246ccd5ab718a160f03e3dbbd1280ebb5389d3637946f73f128af10995288538 records: 67 DB size: 204     -days ago: 12      -date 2023-03-24 15:58:12     -exit? False
ticker 3 proposal 0xde251686e7f312a5e524ba45f4115a887e31ab6a3cb5c2400a0f273d4db2a9c5 records: 50 DB size: 254     -days ago: 14      -date 2023-03-22 10:19:54     -exit? False
ticker 4 proposal 0x58a7bc957b83e4a6424cb72d80bf3c3b0bd8025d0da11d4717fb0a657eafcbd8 records: 68 DB size: 322     -days ago: 14      -date 2023-03-22 02:48:08     -exit? False
ticker 5 proposal 0x0fbdc3e8adbda28970646743492623e92e3a7c2336d453a07d6a090089c72ecf records: 76 DB size: 398     -days ago

In [ ]:
votesDb.head(100)

In [ ]:
votesDb.rename(columns={"createdDate": "voteDate"}, inplace = True)
votesDb.drop_duplicates(inplace=True)
votesDb.drop_duplicates()
votesDb.head(10000)

In [ ]:
#olympus_governance_view.rename(columns={"createdDate": "voteDate"}, inplace = True)
olympus_governance_view.head(200)

In [ ]:
governance_data = pd.merge(votesDb, olympus_governance_view, how='inner', left_on='proposals_id', right_on='proposals_id')
governance_data.head(10)

In [ ]:
crunch_data = db.query("select "
                           "A.proposals_id"
                       "    ,A.startDate "
                       "    ,A.proposals_title "
                       "    ,A.votes_voter "
                       "    ,A.votes_vp "
                       "    ,A.voteDate "
                           ",sum(A.votes_vp) over (Partition by proposals_id  order by votes_vp desc, voteDate asc) as cumulative_vp "
                           ",sum(A.votes_vp) over (Partition by proposals_id) as total_vp "
                           ",(votes_vp::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as percentage_of_total_vp "
                           ",((sum(A.votes_vp) over (Partition by proposals_id  order by votes_vp desc, voteDate asc))::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as cum_percentage_of_total_vp "
                       ",round((sum(A.votes_vp) over (Partition by proposals_id  order by votes_vp desc, voteDate asc))::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as cum_percentage_of_total_vp_stepped "
                           ",row_number() over (Partition by proposals_id order by votes_vp desc, voteDate asc) as proposal_voter_rank "

                           ",(count(*) over (Partition by proposals_id))::decimal total_voters "

                           ",(count(*) over (Partition by proposals_id  order by votes_vp desc, voteDate asc))::decimal/(count(*) over (Partition by proposals_id))::decimal percentage_voters_counted "

                           ",round(100*(count(*) over (Partition by proposals_id  order by votes_vp desc, voteDate asc))::decimal/(count(*) over (Partition by proposals_id)))::decimal percentage_voters_counted_stepped "
                       "from "
                       "    governance_data  A "
                       #"where   to_timestamp((votes_Created::bigint))<'2023-01-01' "
                       ""
                       "Group by "
                           "A.proposals_id"
                       "    ,A.startDate "
                       "    ,A.proposals_title "
                       "    ,A.votes_voter "
                       "    ,A.votes_vp "
                       "    ,A.voteDate "
                       ""
                       "Order by "
                       "    A.startDate  asc "
                       "    , votes_vp desc "
                       "    , voteDate asc"
                       "").df()


crunch_data.insert(0, 'DAO', spacename)
crunch_data.drop_duplicates
crunch_data.head(100)


In [ ]:
#leaders = crunch_data.loc[crunch_data['proposal_voter_rank'] <=3]
#leader_count = leaders.votes_voter.nunique()
#leader_count
#DONT ASK ME WHY THIS DIDNT WORK. HAVE TO DO IT THE UGLY WAY

leader_ranks = db.query("with leader_ranks as "
                        "(Select distinct "
                        "   B.proposals_id"
                        "   ,B.votes_voter"
                        "   ,B.proposal_voter_rank "
                        "   ,(B.proposal_voter_rank +1) as leader_rank "
                        "From "
                        "   (select "
                               "proposals_id"
                               ",votes_voter "
                               ",votes_choice"
                               ",votes_vp"
                               ",votes_created  "
                               ",sum(votes_vp) over (Partition by proposals_id  order by votes_vp desc, votes_created asc) as cumulative_vp"
                               ",sum(votes_vp) over (Partition by proposals_id) as total_vp"
                               ",(votes_vp::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as percentage_of_total_vp "
                               ",((sum(votes_vp) over (Partition by proposals_id  order by votes_vp desc, votes_created asc))::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as cum_percentage_of_total_vp "
                           "    ,round((sum(votes_vp) over (Partition by proposals_id  order by votes_vp desc, votes_created asc))::decimal/sum(votes_vp::decimal) over (Partition by proposals_id)) as cum_percentage_of_total_vp_stepped "
                               ",row_number() over (Partition by proposals_id order by votes_vp desc, votes_created asc) as proposal_voter_rank "
                               ",count(votes_voter) over (Partition by proposals_id  order by votes_vp desc, votes_created asc) total_voters "
                               ",(count(*) over (Partition by proposals_id  order by votes_vp desc, votes_created asc))::decimal/(count(*) over (Partition by proposals_id))::decimal percentage_voters_counted "
                               ",round(100*(count(*) over (Partition by proposals_id  order by votes_vp desc, votes_created asc))::decimal/(count(*) over (Partition by proposals_id)))::decimal percentage_voters_counted_stepped "
                           "from "
                           "    governance_data  "
                               ""
                           "Group by "
                           "    proposals_id"
                           "    ,votes_voter"
                           "    ,votes_choice"
                           "    , votes_vp "
                           "    , votes_created "
                           ""
                           "Order by "
                           "    proposals_id "
                           "    ,votes_vp desc "
                           "    , votes_created asc) B "
                        "where "
                        "   B.cum_percentage_of_total_vp<=0.5) "
                        ""
                        "Select "
                        "   *"
                        "From crunch_data A"
                        "   Join leader_ranks B on A.proposal_voter_rank = B.leader_rank and A.proposals_id = B.proposals_id"
                        ""
                    ).df()
leader_ranks


In [ ]:
dao_members = crunch_data.groupby('DAO').votes_voter.nunique()
dao_members = dao_members.iloc[0]
leader_count =leader_ranks.votes_voter.nunique()
elite = round((leader_count)/(dao_members),4)

print(dao_members, "{0:.2%}".format(elite))


In [ ]:
curve_data = db.query(
    "select "
    "   percentage_voters_counted_stepped "
    "   , avg(percentage_voters_counted) avg_percentage_voters_counted "
    "   , avg(cum_percentage_of_total_vp) avg_percentage_of_total_vp "
    "   ,'Grand Average' as proposal "
    "from crunch_data "
    " group by 1 "
    "UNION ALL "
    "SELECT "
    "   percentage_voters_counted_stepped "
    "   ,percentage_voters_counted "
    "   ,cum_percentage_of_total_vp "
    "   ,proposals_id "
    "FROM crunch_data "
     ).df()

curve_data.head(200)



In [ ]:
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

In [ ]:
path =file+'/'+spacename+'_olympus_governance_data_'+str(date.today().strftime("%b-%d-%Y"))+'.csv'
governance_data.to_csv(path, index = False)

In [ ]:
path =file+'/'+spacename+'_olympus_crunch_data_'+str(date.today().strftime("%b-%d-%Y"))+'.csv'
crunch_data.to_csv(path, index = False)

In [ ]:
path =file+'/'+spacename+'_olympus_curve_data_'+str(date.today().strftime("%b-%d-%Y"))+'.csv'
curve_data.to_csv(path, index = False)

In [ ]:
plt.rc("figure", figsize=(40, 20))
#sns.set_style("whitegrid")
plt.rc("font", size=25)
data_means = crunch_data.groupby("percentage_voters_counted_stepped")["cum_percentage_of_total_vp","percentage_voters_counted"].agg("mean").reset_index()
##print(data_means)
plot_title = spacename + ' snapshots % of vote along population with Average as X'

ax=sns.scatterplot(data=crunch_data, hue = "proposals_id",y="cum_percentage_of_total_vp",x="percentage_voters_counted_stepped").set(title=plot_title,xlabel='% of voters',ylabel='% of voting power')
chart = sns.scatterplot(data=data_means,x="percentage_voters_counted_stepped",y="cum_percentage_of_total_vp",zorder=3, s=800,marker='X',color = 'orange')
plt.legend(bbox_to_anchor=(1.02, 0.99), loc='upper left', borderaxespad=0)
#and save the chart file, too
print(chart)
plt.savefig(file+'\\'+spacename+' vote power distribution.png',transparent =False,  dpi=100)